<a href="https://colab.research.google.com/github/Nouman97/Computer-Vision/blob/master/Trash_Classification_using_Mobile_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
from PIL import Image
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 224, 224

Using TensorFlow backend.


In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_data_dir = "/content/gdrive/My Drive/SVM_Data/Train/"
validation_data_dir = "/content/gdrive/My Drive/SVM_Data/Validation/"#"drive/tertiary/validation"

In [0]:
mobile = applications.mobilenet.MobileNet()

In [0]:
mobile.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [0]:
x = mobile.layers[-6].output
predictions = Dense(2,activation='softmax')(x)
model = Model(inputs = mobile.input,outputs = predictions)

for layer in model.layers:
  layer.trainable = True
  
Adam = optimizers.adam(lr=0.0001)
sgd = optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0005, nesterov=False)
model.compile(optimizer = Adam,loss = "binary_crossentropy", metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(preprocessing_function=applications.mobilenet.preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=applications.mobilenet.preprocess_input)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (224, 224),
batch_size = 32)

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size = (224, 224),batch_size=15)

Found 2000 images belonging to 2 classes.
Found 810 images belonging to 2 classes.


In [0]:
model.fit_generator(train_generator,validation_data=validation_generator,epochs=5,steps_per_epoch = 23,validation_steps=12)

Epoch 1/5
23/23 [==============================] - 398s 17s/step - loss: 0.1049 - acc: 0.9552 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 2/5
23/23 [==============================] - 480s 21s/step - loss: 9.3953e-04 - acc: 1.0000 - val_loss: 5.2730e-05 - val_acc: 1.0000
Epoch 3/5
23/23 [==============================] - 464s 20s/step - loss: 1.7910e-04 - acc: 1.0000 - val_loss: 3.1395e-05 - val_acc: 1.0000
Epoch 4/5
23/23 [==============================] - 376s 16s/step - loss: 1.4711e-04 - acc: 1.0000 - val_loss: 6.1524e-05 - val_acc: 1.0000
Epoch 5/5
23/23 [==============================] - 384s 17s/step - loss: 3.5844e-04 - acc: 1.0000 - val_loss: 2.7693e-05 - val_acc: 1.0000


In [0]:
model.save_weights('mobilenet-tertiary.h5')